# Equally-weighted portfolio of S&P500

In [13]:
import numpy as np 
import pandas as pd 
import requests
from datetime import datetime
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

In [3]:
# import list of s&p500 stocks
snp_list = pd.read_csv("sp_500_stocks.csv")

In [4]:
snp_list.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


In [121]:
# copy to new dataframe
equal_weight = snp_list.copy()

In [122]:
equal_weight

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [8]:
# get api key (str)
from secrets import IEX_CLOUD_API_TOKEN

In [123]:
# set column names
col_names = ['Price', 'Market Cap', 'Number of shares to buy']

## Method 1 : call api for one symbol at a time (very slow)

In [124]:
# allocate None to all (exept Ticker)
equal_weight[col_names] = None

In [125]:
equal_weight

,Ticker,Price,Market Cap,Number of shares to buy
0,A,None,None,None
1,AAL,None,None,None
2,AAP,None,None,None
3,AAPL,None,None,None
4,ABBV,None,None,None
...,...,...,...,...
500,YUM,None,None,None
501,ZBH,None,None,None
502,ZBRA,None,None,None
503,ZION,None,None,None


In [126]:
def api_price(symbol):
    """
        get 'latestPrice' for one symbol
    """
    api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/latestPrice?token={IEX_CLOUD_API_TOKEN}"
    return requests.get(api_url).json()

In [127]:
def api_mkt_cap(symbol):
    """
        get 'marketCap' for one symbol
    """
    api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/marketCap?token={IEX_CLOUD_API_TOKEN}"
    return requests.get(api_url).json()

In [128]:
# just for looping convenient
stock_attribs = {
    'Price': api_price,
    'Market Cap': api_mkt_cap
}

In [144]:
# timing
d0 = datetime.now()

for key in tqdm_notebook(stock_attribs.keys()):
    # progress_apply ==> like pandas.apply but printing progress bar
    equal_weight[key] = equal_weight['Ticker'].progress_apply(stock_attribs[key])

# print time elapsed
print(datetime.now() - d0)

C:\Users\kfuan\Anaconda3\envs\dl_fastai\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel




0:17:20.673721


### 17 mins to call all symbols !!!!!!

In [145]:
equal_weight.head(15)

,Ticker,Price,Market Cap,Number of shares to buy
0,A,125.85,37774558980,None
1,AAL,18.10,11060560341,None
2,AAP,161.60,10580504976,None
3,AAPL,137.51,2353649615704,None
4,ABBV,111.11,193486010360,None
5,ABC,106.56,22277206566,None
6,ABMD,333.52,14760553748,None
7,ABT,122.57,214681304258,None
8,ACN,255.78,172342773345,None
9,ADBE,502.98,234617382882,None


## Method 2 : use batch api (maximum call 100 symbols at a time) (a lot faster)

In [152]:
def chunks(input_list, n):
    """
        spilt data into chunks of length n
    """
    for i in range(0, len(input_list), n):
        yield input_list[i: i+n]

In [287]:
def batch_api(batch_symbol):
    """
        calling api (maximum of 100 symbols)
        'latestPrice' , 'marketCap'
    """
    api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&filter=latestPrice,marketCap&symbols={batch_symbol}&token={IEX_CLOUD_API_TOKEN}'
    return requests.get(api_url).json()

In [261]:
def get_price(x):
    """
        easily extract lastestPrice from .json()
    """
    return x['quote']['latestPrice']

def get_mkt_cap(x):
    """
        easily extract marketCap from .json()
    """
    return x['quote']['marketCap']

In [306]:
data

{'YUM': {'quote': {'latestPrice': 108.52, 'marketCap': 32308202931}},
 'ZBH': {'quote': {'latestPrice': 162.25, 'marketCap': 34113685223}},
 'ZBRA': {'quote': {'latestPrice': 422.93, 'marketCap': 21641109806}},
 'ZION': {'quote': {'latestPrice': 48.23, 'marketCap': 7840984298}},
 'ZTS': {'quote': {'latestPrice': 157.58, 'marketCap': 73843502568}}}

In [305]:
data_array

array([{'quote': {'latestPrice': 108.52, 'marketCap': 32308202931}},
       {'quote': {'latestPrice': 162.25, 'marketCap': 34113685223}},
       {'quote': {'latestPrice': 422.93, 'marketCap': 21641109806}},
       {'quote': {'latestPrice': 48.23, 'marketCap': 7840984298}},
       {'quote': {'latestPrice': 157.58, 'marketCap': 73843502568}}],
      dtype=object)

In [263]:
# vectorize np.array to return all values at once
vec_price = np.vectorize(get_price)
vec_mkt_cap = np.vectorize(get_mkt_cap)

In [290]:
# timing
d0 = datetime.now()

batch_size = 100
#chunks of symbols
stock_batches = np.array(list(chunks(batch_equal_weight['Ticker'].values, batch_size)))

batch_strings = []
# transform each chunks into one string
for i in tqdm_notebook(range(0, len(stock_batches))):
    batch_strings.append(','.join(stock_batches[i]))

start = 0
for batch in tqdm_notebook(batch_strings):
    # get api data
    data = batch_api(batch)
    # get dictvalues ==> into list ==> into numpy.array
    data_array = np.array(list(data.values()))
    
    # apply vectorize function to numpy.array
    prices = vec_price(data_array)
    mkt_caps = vec_mkt_cap(data_array)
    
    # add data to dataframe (100 at a time, execpt last batch)
    batch_equal_weight['Price'][start: start + len(data_array)] = prices
    batch_equal_weight['Market Cap'][start: start + len(data_array)] = mkt_caps
    
    # shifting starting row each time
    start += len(data_array)

# print time elapsed
print(datetime.now() - d0)

<ipython-input-290-3d7619e73af4>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  stock_batches = np.array(list(chunks(batch_equal_weight['Ticker'].values, batch_size)))



0:00:06.287925


### 6 secs lmao !!!!!

In [291]:
batch_equal_weight

,Ticker,Price,Market Cap,Number of shares to buy
0,A,125.75,37517953803,None
1,AAL,18.1,11113829497,None
2,AAP,156.25,10711987136,None
3,AAPL,136.2,2306634914794,None
4,ABBV,109.84,197725190496,None
...,...,...,...,...
500,YUM,108.52,32308202931,None
501,ZBH,162.25,34113685223,None
502,ZBRA,422.93,21641109806,None
503,ZION,48.23,7840984298,None


In [302]:
# asking user to input size of portfolio
while True:
    try:
        portfolio_size = float(input("Enter value of your portfolio: "))
        break
    except:
        print('Please enter the valid value.')
        continue
        
# equal-weighted ==> position size = portfolio size / number of assets
position_size = portfolio_size / batch_equal_weight.shape[0]
# number of each stock ==> position size / price
batch_equal_weight['Number of shares to buy'] = np.floor(position_size / batch_equal_weight['Price'])

display(batch_equal_weight.head())
display(batch_equal_weight.tail())

Enter value of your portfolio: 10000000


,Ticker,Price,Market Cap,Number of shares to buy
0,A,125.75,37517953803,157
1,AAL,18.1,11113829497,1094
2,AAP,156.25,10711987136,126
3,AAPL,136.2,2306634914794,145
4,ABBV,109.84,197725190496,180


,Ticker,Price,Market Cap,Number of shares to buy
500,YUM,108.52,32308202931,182
501,ZBH,162.25,34113685223,122
502,ZBRA,422.93,21641109806,46
503,ZION,48.23,7840984298,410
504,ZTS,157.58,73843502568,125


In [304]:
# save file to excel
batch_equal_weight.to_excel('equal weighted S&P500.xlsx',
                            sheet_name='Equal Weighted S&P500',
                            index=False)